<a href="https://colab.research.google.com/github/castorini/anserini-notebooks/blob/master/solrini_msmarco_passage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%capture
!apt-get install maven -qq

In [0]:
%%capture
!git clone https://github.com/castorini/anserini.git
%cd anserini
!mvn clean package appassembler:assemble -DskipTests -Dmaven.javadoc.skip=true
!cd eval && tar xvfz trec_eval.9.0.4.tar.gz && cd trec_eval.9.0.4 && make

In [3]:
!ls target

anserini-0.7.3-SNAPSHOT-fatjar.jar  classes		    maven-archiver
anserini-0.7.3-SNAPSHOT.jar	    generated-sources	    maven-status
appassembler			    generated-test-sources  test-classes


In [0]:
%%capture
!wget https://archive.apache.org/dist/lucene/solr/8.3.0/solr-8.3.0.tgz
!mkdir solrini && tar -zxvf solr*.tgz -C solrini --strip-components=1

In [5]:
!solrini/bin/solr start -c -m 8G -force

Waiting up to 180 seconds to see Solr running on port 8983 [|]   [/]   [-]   [\]   [|]   [/]   [-]   [\]   [|]   [/]   [-]   [\]   [|]   [/]   [-]   [\]   [|]   [/]  
Started Solr server on port 8983 (pid=840). Happy searching!

    

In [6]:
!python src/main/python/run_solr_regression.py --ping

2020-03-03 20:45:32,293 INFO - Pinging Solr instance...
2020-03-03 20:45:32,421 INFO - ... appears to alive! :)


In [0]:
%%capture
!mkdir msmarco-passage
!wget https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz -P msmarco-passage
!tar -xzvf msmarco-passage/collectionandqueries.tar.gz -C msmarco-passage

In [0]:
%%capture
!python ./src/main/python/msmarco/convert_collection_to_jsonl.py \
 --collection_path msmarco-passage/collection.tsv --output_folder msmarco-passage/collection_jsonl

In [9]:
!solrini/bin/solr create -n anserini -c msmarco-passage -force

Created collection 'msmarco-passage' with 1 shard(s), 1 replica(s) with config-set 'anserini'


In [10]:
!curl http://localhost:8983/solr/admin/collections?action=LIST

{
  "responseHeader":{
    "status":0,
    "QTime":0},
  "collections":["msmarco-passage"]}


In [11]:
!sh target/appassembler/bin/IndexCollection -collection JsonCollection -generator JsoupGenerator \
   -threads 8 -input msmarco-passage/collection_jsonl \
   -solr -solr.index msmarco-passage -solr.zkUrl localhost:9983 \
   -storePositions -storeDocvectors -storeTransformedDocs

2020-03-03 20:54:02,968 INFO  [main] index.IndexCollection (IndexCollection.java:606) - Setting log level to INFO
2020-03-03 20:54:02,972 INFO  [main] index.IndexCollection (IndexCollection.java:609) - Starting indexer...
2020-03-03 20:54:02,973 INFO  [main] index.IndexCollection (IndexCollection.java:610) - ============ Loading Parameters ============
2020-03-03 20:54:02,979 INFO  [main] index.IndexCollection (IndexCollection.java:611) - DocumentCollection path: msmarco-passage/collection_jsonl
2020-03-03 20:54:02,979 INFO  [main] index.IndexCollection (IndexCollection.java:612) - CollectionClass: JsonCollection
2020-03-03 20:54:02,981 INFO  [main] index.IndexCollection (IndexCollection.java:613) - Generator: JsoupGenerator
2020-03-03 20:54:02,982 INFO  [main] index.IndexCollection (IndexCollection.java:614) - Threads: 8
2020-03-03 20:54:02,984 INFO  [main] index.IndexCollection (IndexCollection.java:615) - Stemmer: porter
2020-03-03 20:54:02,985 INFO  [main] index.IndexCollection (In

In [0]:
%%capture
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

In [0]:
get_ipython().system_raw('./ngrok http 8983 &')

In [14]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://a0cc3cef.ngrok.io
